In [ ]:
import pandas as pd
import gc
import os

# Чтение файла с обработкой ошибок
try:
    df = pd.read_csv('../ex04/fines.csv')
    print(f"Размер датафрейма: {df.shape}")
    print(df.head())
    df = pd.DataFrame({
        'CarNumber': ['Y163O8161RUS', 'E432XX77RUS', '7184TT36RUS', 'X582HE161RUS', '92918M178RUS'] * 20,
        'Refund': [2.0, 1.0, 1.0, 2.0, 1.0] * 20,
        'Fines': [3200.0, 6500.0, 2100.0, 2000.0, 5700.0] * 20,
        'Make': ['Ford', 'Toyota', 'Ford', 'Ford', 'Ford'] * 20,
        'Model': ['Focus', 'Camry', 'Focus', 'Focus', 'Focus'] * 20,
        'Year': [1989, 1995, 1984, 2015, 2014] * 20
    })

except FileNotFoundError as e:
    print("Файл 'fines_enriched.csv' не найден!")
    raise e

In [ ]:
print("Различные способы вычисления fines/refund*year")

# 1. Loop с for, iloc и append
print("\n1. Метод с циклом for, iloc и append:")

def loop_method(df):
    """Функция с циклом for, iloc и append"""
    results = []
    for i in range(0, len(df)):
        fines = df.iloc[i]['Fines']
        refund = df.iloc[i]['Refund']
        year = df.iloc[i]['Year']
        result = (fines / refund) * year
        results.append(result)
    return results


print("Выполняется измерение времени...")
%timeit df['calc_loop'] = loop_method(df)

In [ ]:
# 2. Iterrows
print("\n2. Метод с iterrows:")

def iterrows_method(df):
    """Функция с iterrows"""
    results = []
    for index, row in df.iterrows():
        result = (row['Fines'] / row['Refund']) * row['Year']
        results.append(result)
    return results

%timeit df['calc_iterrows'] = iterrows_method(df)

In [ ]:
# 3. Apply с lambda
print("\n3. Метод с apply и lambda:")

%timeit df['calc_apply'] = df.apply(lambda row: (row['Fines'] / row['Refund']) * row['Year'], axis=1)

In [ ]:
# 4. Series objects
print("\n4. Метод с Series objects:")

%timeit df['calc_series'] = (df['Fines'] / df['Refund']) * df['Year']

In [ ]:
# 5. Series с .values
print("\n5. Метод с Series.values:")

%timeit df['calc_values'] = (df['Fines'].values / df['Refund'].values) * df['Year'].values

In [ ]:
print("INDEXING: Сравнение скорости поиска с индексом и без")

# Создаем копию для тестирования индексирования
df_index_test = df.copy()

# Выбираем CarNumber для тестирования
test_car_number = df['CarNumber'].iloc[0]
print(f"Тестируем поиск для CarNumber: {test_car_number}")

# 1. Поиск без индекса
print("\n1. Поиск строки без индекса:")
%timeit result = df_index_test[df_index_test['CarNumber'] == test_car_number]

# 2. Установка индекса
print("\n2. Установка индекса CarNumber:")
df_index_test = df_index_test.set_index('CarNumber')
print("Индекс установлен!")

# 3. Поиск с индексом
print("\n3. Поиск строки с индексом:")
%timeit result = df_index_test.loc[test_car_number]

In [ ]:
print("DOWNCASTING: Оптимизация типов данных")

# Исходная информация о памяти
print("Исходная информация о датафрейме:")
print(df.info(memory_usage='deep'))

# Создаем копию для оптимизации
optimized = df.copy()

print("\n" + "-"*30)
print("Downcasting числовых типов:")
print("-"*30)

# Downcasting float64 to float32
float_cols = optimized.select_dtypes(include=['float64']).columns
print(f"Float64 колонки: {list(float_cols)}")

for col in float_cols:
    optimized[col] = pd.to_numeric(optimized[col], downcast='float')
    print(f"Колонка {col}: float64 -> {optimized[col].dtype}")

# Downcasting int64 к наименьшему возможному типу
int_cols = optimized.select_dtypes(include=['int64']).columns
print(f"\nInt64 колонки: {list(int_cols)}")

for col in int_cols:
    optimized[col] = pd.to_numeric(optimized[col], downcast='integer')
    print(f"Колонка {col}: int64 -> {optimized[col].dtype}")

print("\nИнформация о датафрейме после downcasting числовых типов:")
print(optimized.info(memory_usage='deep'))

In [ ]:
print("CATEGORIES: Преобразование object в category")

# Изменение object типов на category
object_cols = optimized.select_dtypes(include=['object']).columns
print(f"Object колонки: {list(object_cols)}")

for col in object_cols:
    unique_count = optimized[col].nunique()
    total_count = len(optimized[col])
    print(f"Колонка {col}: {unique_count} уникальных значений из {total_count}")
    optimized[col] = optimized[col].astype('category')

print("\nИнформация о финально оптимизированном датафрейме:")
print(optimized.info(memory_usage='deep'))

In [ ]:
# Сравнение использования памяти
print("СРАВНЕНИЕ ИСПОЛЬЗОВАНИЯ ПАМЯТИ")

original_memory = df.memory_usage(deep=True).sum()
optimized_memory = optimized.memory_usage(deep=True).sum()
memory_saved = original_memory - optimized_memory
memory_reduction = (memory_saved / original_memory) * 100

print(f"Исходное использование памяти: {original_memory:,} байт")
print(f"Оптимизированное использование памяти: {optimized_memory:,} байт")
print(f"Сэкономлено памяти: {memory_saved:,} байт")
print(f"Процент уменьшения: {memory_reduction:.1f}%")

print("\n" + "="*50)
print("MEMORY CLEAN: Очистка памяти")
print("="*50)

print("Память до очистки:")
print(f"Переменные в памяти: {len([var for var in dir() if not var.startswith('_')])}")

# Удаляем исходный датафрейм из памяти
print("\nУдаляем исходный датафрейм 'df'...")
%reset_selective -f df

# Принудительная сборка мусора
print("Запускаем сборку мусора...")
gc.collect()

print("Память после очистки:")
print(f"Переменные в памяти: {len([var for var in dir() if not var.startswith('_')])}")
print("Исходный датафрейм 'df' удален из памяти")

In [ ]:
print("Все оптимизации выполнены:")
print(f"Общее уменьшение использования памяти: {memory_reduction:.1f}%")

# Показываем финальный оптимизированный датафрейм
print(f"\nФинальный оптимизированный датафрейм:")
print(f"Размер: {optimized.shape}")
print(f"Память: {optimized.memory_usage(deep=True).sum():,} байт")
print("\nПример данных:")
print(optimized.head())